In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!pip install category_encoders

     |████████████████████████████████| 86 kB 3.2 MB/s 


In [4]:
import pandas as pd

df_data = pd.read_csv('/content/heart_2020_cleaned.csv')
df_data

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319790,Yes,27.41,Yes,No,No,7.0,0.0,Yes,Male,60-64,Hispanic,Yes,No,Fair,6.0,Yes,No,No
319791,No,29.84,Yes,No,No,0.0,0.0,No,Male,35-39,Hispanic,No,Yes,Very good,5.0,Yes,No,No
319792,No,24.24,No,No,No,0.0,0.0,No,Female,45-49,Hispanic,No,Yes,Good,6.0,No,No,No
319793,No,32.81,No,No,No,0.0,0.0,No,Female,25-29,Hispanic,No,No,Good,12.0,No,No,No


In [5]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319795 entries, 0 to 319794
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   HeartDisease      319795 non-null  object 
 1   BMI               319795 non-null  float64
 2   Smoking           319795 non-null  object 
 3   AlcoholDrinking   319795 non-null  object 
 4   Stroke            319795 non-null  object 
 5   PhysicalHealth    319795 non-null  float64
 6   MentalHealth      319795 non-null  float64
 7   DiffWalking       319795 non-null  object 
 8   Sex               319795 non-null  object 
 9   AgeCategory       319795 non-null  object 
 10  Race              319795 non-null  object 
 11  Diabetic          319795 non-null  object 
 12  PhysicalActivity  319795 non-null  object 
 13  GenHealth         319795 non-null  object 
 14  SleepTime         319795 non-null  float64
 15  Asthma            319795 non-null  object 
 16  KidneyDisease     31

In [9]:
df_data['HeartDisease'] = df_data['HeartDisease'].str[0] == 'Y'

AttributeError: ignored

In [11]:
df_data['HeartDisease'].value_counts(normalize=True)

False    0.914405
True     0.085595
Name: HeartDisease, dtype: float64

In [12]:
from sklearn.model_selection import train_test_split

target = df_data['HeartDisease']
data = df_data.drop('HeartDisease', axis=1)

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, stratify=target)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train)



In [62]:
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(n_jobs=-1, oob_score=True, class_weight="balanced")
)

In [63]:
from sklearn.model_selection import RandomizedSearchCV

dists = {
    # 'randomforestclassifier__n_estimators':range(10, 300),
    'randomforestclassifier__max_depth':range(1, 15),
    'randomforestclassifier__min_samples_split':range(2, 30),
    'randomforestclassifier__min_samples_leaf':range(1, 20)
}

randomCV = RandomizedSearchCV(
    pipe, 
    param_distributions=dists, 
    n_iter=30, 
    cv=3,
    scoring='f1',
    verbose=1,
    n_jobs=-1
)

randomCV.fit(X_train, y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('ordinalencoder',
                                              OrdinalEncoder()),
                                             ('simpleimputer', SimpleImputer()),
                                             ('randomforestclassifier',
                                              RandomForestClassifier(class_weight='balanced',
                                                                     n_jobs=-1,
                                                                     oob_score=True))]),
                   n_iter=30, n_jobs=-1,
                   param_distributions={'randomforestclassifier__max_depth': range(1, 15),
                                        'randomforestclassifier__min_samples_leaf': range(1, 20),
                                        'randomforestclassifier__min_samples_split': range(2, 30)},
                   scoring='f1', verbose=1)

In [77]:
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
import numpy as np

pipe_best = randomCV.best_estimator_
pred_prob = pipe_best.predict_proba(X_val)[:,1]

fpr, tpr, thresh = roc_curve(y_val, pred_prob)

optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresh[optimal_idx]
y_pred_optimal = pred_prob >= optimal_threshold
print(classification_report(y_val, y_pred_optimal))
y_pred = pipe_best.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

       False       0.97      0.74      0.84     46788
        True       0.22      0.79      0.34      4380

    accuracy                           0.74     51168
   macro avg       0.60      0.76      0.59     51168
weighted avg       0.91      0.74      0.80     51168

              precision    recall  f1-score   support

       False       0.97      0.79      0.87     46788
        True       0.24      0.71      0.36      4380

    accuracy                           0.79     51168
   macro avg       0.60      0.75      0.62     51168
weighted avg       0.90      0.79      0.83     51168



In [81]:
df_temp = pd.DataFrame(columns=X_val.columns)
df_temp.loc[0] = [40.0,"Yes","Yes","Yes",20.0,20.0,"No","Male","35-39","Hispanic","No","No","Fair",6.0,"No","No","Yes"]
pipe_best.predict(df_temp)[0]

True

In [79]:
import pickle

with open('/content/model.pkl','wb') as pickle_file:
    pickle.dump(pipe_best, pickle_file)

In [80]:
X_val.columns

Index(['BMI', 'Smoking', 'AlcoholDrinking', 'Stroke', 'PhysicalHealth',
       'MentalHealth', 'DiffWalking', 'Sex', 'AgeCategory', 'Race', 'Diabetic',
       'PhysicalActivity', 'GenHealth', 'SleepTime', 'Asthma', 'KidneyDisease',
       'SkinCancer'],
      dtype='object')